In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src/')

import utils, split, merge, document_analysis
from split import split_merge

In [3]:
import logging
import os
import math
import io
import json
import PyPDF2


In [4]:
def run_document_detection(file_name):
    job_id = document_analysis.start_job(merge.client, utils.BUCKET, file_name)
    logging.info("Started job with id: {}".format(job_id))

    if document_analysis.is_job_complete(merge.client, job_id):
        response = document_analysis.get_job_results(merge.client, job_id)

    return response

In [5]:
# Textract hard limits
current_async_page_limit = 3000
current_async_size_limit = 524288000

# specify the file name as it appears in s3
file_name = "test_example2.pdf"
input_file = io.BytesIO(utils.read_file_from_s3(file_name))

logging.getLogger().setLevel(logging.INFO)

In [6]:
# same as the main function in main.py
def run(current_async_page_limit, current_async_size_limit):
    textract_response = {}

    try:
        if file_name.split(".")[1] == "pdf":
            reader = PyPDF2.PdfFileReader(input_file)
            textract_response = {}
            if reader.getNumPages() > current_async_page_limit:
                logging.info("Page size exceeds textract hard limit. Starting workaround")
                while True:
                    current_async_page_limit, textract_response = split_merge(
                        file_name,
                        reader,
                        current_async_page_limit,
                        current_async_size_limit,
                    )
                    if len(textract_response) > 0:
                        break

            elif input_file.getbuffer().nbytes > current_async_size_limit:
                page_limit = math.ceil(reader.getNumPages() / 2)
                logging.info("Document size exceeds textract hard limit. Starting workaround")
                while True:
                    page_limit, textract_response = split_merge(
                        file_name,
                        reader,
                        page_limit,
                        current_async_size_limit,
                    )
                    if len(textract_response) > 0:
                        break
            else:
                logging.info("Document within textract hard limits. Workaround not required")
                textract_response = run_document_detection(file_name)
            
            with open(f'{file_name.split(".")[0]}_data.json', 'w', encoding='utf-8') as f:
                json.dump(textract_response, f, ensure_ascii=False, indent=4)
            return textract_response 
        else:
            raise ValueError("Expected a PDF file as input")
    except Exception as e:
        print(e)

In [7]:
%%timeit -r 1 -n 1
textract_response = run(current_async_page_limit, current_async_size_limit)
print(F"Number of text blocks found:{len(textract_response[0]['Blocks'])}")

INFO:root:Document within textract hard limits. Workaround not required
INFO:root:Started job with id: 0147f79d746b1afc2ba785b9f5fbe4fc95bd238b0f09c66902a13f159ab6a630
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: SUCCEEDED


Number of text blocks found:795
20.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%%timeit -r 1 -n 1
# Simulate an artificial hard limit
textract_response = run(3, current_async_size_limit)
print(F"Number of text blocks found:{len(textract_response[0]['Blocks'])}")

INFO:root:Page size exceeds textract hard limit. Starting workaround
INFO:root:Started job with id: 54e5c4ac8627c12644df0e6d7329a7898ae44eb220de710975b6cfcc995a58f2
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: SUCCEEDED
INFO:root:Started job with id: 449aa3c2330af4f50c4bdbc503f58728a912ff31bf82935361f5a2b2ac395ebc
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: SUCCEEDED


Number of text blocks found:795
31.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
